In [1]:
import sqlite3
import os

## **CREATE TABLES**
--------------------------------------------

In [2]:
#create path
database_path = os.path.join(os.pardir, "databases", "image_recommender.db")
database_dir = os.path.dirname(database_path)

if not os.path.exists(database_dir):
    os.makedirs(database_dir)

conn = sqlite3.connect(database_path)
curs = conn.cursor()

In [3]:
#delete table if needed 
table_name = ["images", "scores"]

for name in table_name:
    curs.execute(f"DROP TABLE if exists {name}")

In [4]:
#create table paths
create_table = curs.execute("""CREATE TABLE if not exists images
             (imageid INTEGER PRIMARY KEY,
             filepath text not null,
             filename text not null);""")

conn.commit()

In [5]:
#create table similarities

create_table2 = curs.execute("""CREATE TABLE if not exists scores
             (imageid INTEGER PRIMARY KEY,
             color_similarity real,
             embedding_similarity real,
             third_similarity real,
             FOREIGN KEY (imageid) references images(imageid) ON DELETE CASCADE);""")

conn.commit()

In [6]:
# check if table is created 
print(f"Database created: {os.listdir(database_dir)}")

#show database

def describe_table(curs, table_name):
    curs.execute(f"PRAGMA table_info({table_name})")
    return curs.fetchall()


#check which databases are created
curs.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = curs.fetchall()
print(f"Tables: {tables}")

table_structure = describe_table(curs, "images")
table_structure


Database created: ['image_recommender.db']
Tables: [('images',), ('scores',)]


[(0, 'imageid', 'INTEGER', 0, None, 1),
 (1, 'filepath', 'TEXT', 1, None, 0),
 (2, 'filename', 'TEXT', 1, None, 0)]

## **FETCH DATA FROM HARDDRIVE**
-----------------------------------

In [7]:
#check path to harddrive
root_dir = "F:\data\image_data"
if os.path.exists(root_dir):
    print(True)

True


In [8]:
#generator
def load_images_from_harddrive(root_dir):
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith(".jpg", ".png", ".jpeg"):
                yield root, file
                

In [ ]:
for image in load_images_from_harddrive(root_dir):
    print(image)

## **INSERT DATA INTO DATABASE**
--------------------

In [ ]:
#extract and insert


insert_statement = ("""INSERT INTO images ('imageid', 'filepath', 'filename')
             VALUES (?,?,?)""")

#values = (imageid,filepath, filename, color_similarity, embedding_similarity, third_similarity)

curs.execute(insert_statement,values)

conn.commit()